In [4]:
import tvm
from tvm import relay
import numpy as np

In [5]:
x = relay.var('x', shape=(2,))
i = relay.var('i', shape=(), dtype='int32')
s = relay.var('s', shape=(2,))
cond = i < relay.const(10, dtype='int32')

loop = relay.var('while_loop')
sb = relay.scope_builder.ScopeBuilder()
with sb.if_scope(cond):
    ii = i + relay.const(1, dtype='int32')
    ss = s + x
    sb.ret(loop(ii, ss))
with sb.else_scope():
    sb.ret(s)
func = relay.Function([i, s], sb.get())
ret = relay.Let(loop, func, loop(relay.const(0, dtype='int32'), 
                                 relay.zeros(shape=(2,), dtype='float32')))
mod = relay.Module()
mod["main"] = relay.Function([x], ret)
print(mod["main"])
mod = relay.transform.LambdaLift()(mod)

v0.0.4
fn (%x: Tensor[(2), float32]) -> Tensor[(2), float32] {
  let %while_loop-malformed-ir = fn (%i: int32, %s: Tensor[(2), float32]) -> Tensor[(2), float32] {
    %0 = less(%i, 10 /* ty=int32 */) /* ty=bool */;
    if (%0) {
      %1 = add(%i, 1 /* ty=int32 */) /* ty=int32 */;
      %2 = add(%s, %x) /* ty=Tensor[(2), float32] */;
      free_var %while_loop: fn (int32, Tensor[(2), float32]) -> Tensor[(2), float32]
      %while_loop(%1, %2) /* ty=Tensor[(2), float32] */
    } else {
      %s
    }
  };
  %3 = zeros(shape=[2], dtype="float32") /* ty=Tensor[(2), float32] */;
  %while_loop(0 /* ty=int32 */, %3) /* ty=Tensor[(2), float32] */
}


In [7]:
with relay.build_config(opt_level=1):
    ex = relay.create_executor("debug", mod=mod, ctx=tvm.cpu(), target="llvm")
result = ex.evaluate()(np.ones((2,), dtype='float32'))
print(result.asnumpy())

[10. 10.]
